# はじめに

## JijModelingとは

`jijmodeling`は、数学的最適化モデルを直感的に記述するために設計されたPythonパッケージです。その主な特徴は次のとおりです：

- 特定のインスタンスデータなしでモデルを記述できるため、モデルの検証が迅速になり、再利用が容易になります。インスタンスのサイズは、モデルの記述や操作のパフォーマンスに影響を与えません。
- 線形計画法、混合整数計画法、非線形計画法など、さまざまな種類の最適化問題に対する共通のインターフェースとして機能します。
- モデルをプログラム的に操作できるため、モデルを部分ごとに構築したり、制約の構築においてより複雑なロジックを使用したりできます。
- LaTeXへの出力をサポートしています。Jupyterノートブックと組み合わせることで、モデルが期待通りに適合しているかどうかを迅速に対話的に確認できます。

`jijmodeling`は、Pythonコードを使用してモデルを記述するためのツールであり、モデルを評価または解決するものではありません。主な動機は、インスタンスデータに関係なく、モデルの代数構造に焦点を当てることで、モデルを論理的に考察し、検証し、より迅速に変更できるようにすることです。同時に、ソルバーが期待する入力形式を生成するためのステップとしても機能します。

ソルバーと一緒に使用するためには、実際のインスタンスデータと組み合わせて、内部の[JijZeptサービス](https://www.jijzept.com)や、無料で利用できる[jijmodeling-transpiler](https://www.documentation.jijzept.com/docs/jijmodelingtranspiler/)などの別のツールによって特定のソルバーの入力形式に変換する必要があります。

## インストール

`pip`を使用する場合、次のコマンドで`jijmodeling`をインストールできます：

```bash
pip install jijmodeling
```

`jijmodeling`はPython 3.9以上が必要であることに注意してください。

## クイックスタート

`jijmodeling`の使い方をよりよく理解するために、基本的な例を見ていきましょう。まず、シンプルなモデルを作成し、それを変換し、ソルバーで実行する手順を説明します。最初の2つのセクションでは、jijmodelingだけで十分ですが、LaTeX出力を簡単に確認するために[Jupyterノートブック](https://jupyter.org)を使用することをお勧めします。

3つ目のセクションでは、[JijModelingTranspiler](https://www.documentation.jijzept.com/docs/jijmodelingtranspiler/)と[OpenJij](https://openjij.github.io/OpenJij/index.html)を使用します。jijmodelingのモデルは、有料のJijZeptサービスでも使用可能です。JijModelingTranspilerとOpenJijは通常通り`pip`を使用してインストールできます：

```bash
pip install jijmodeling-transpiler openjij
```

### 概要

`jijmodeling`を使用することは、数学的最適化モデルに精通している人にとって自然に感じられるはずです。

さまざまなクラス（`BinaryVar`や`Placeholder`など）を基本的な数学的操作を使用して組み合わせることで、さまざまな数学的表現を表現します。`___Var`クラスは、さまざまな種類の決定変数を指します。`Placeholder`は、後で指定される任意の種類の定数や値を表します。つまり、モデルから抽象化し、インスタンス固有のデータの一部としてマークしたいものです。もちろん、数値リテラルも使用できます。

In [12]:
import jijmodeling as jm

x = jm.BinaryVar("x")
y = jm.IntegerVar("y", lower_bound=1, upper_bound=10)
n = jm.Placeholder("n")
exp = x * (y ** 2) * n

上記のプレースホルダーと変数はゼロ次元（スカラー）ですが、これらのクラスを使用して配列や多次元変数および定数を表現することもできます（後で詳しく説明します）。

`jijmodeling`では、このような式を`Problem`オブジェクトに追加することでモデルを構築し、モデル全体を表現します。制約は、比較表現をラップする`Constraint`クラスによって定義されます（`Constraint`では`<=`、`==`、および`>=`のみがサポートされています）。

In [13]:
a = jm.IntegerVar("a", lower_bound=5, upper_bound=20)
b = jm.IntegerVar("b", lower_bound=1, upper_bound=20)
c = jm.IntegerVar("c", lower_bound=20, upper_bound=30)
n = jm.Placeholder("n")

problem = jm.Problem("my problem")
problem += n * (a + b + c)
problem += jm.Constraint("c1", 2 * (b + c) <= 75)
problem += jm.Constraint("c2", a + b <= 40)
problem

### モデルの作成

一般的な[ナップサック問題](https://www.documentation.jijzept.com/docs/tutorial/knapsack)をどのようにモデル化するか見てみましょう。

この問題では、$N$個の貴重なアイテムがあります。できるだけ多くのアイテムを持ち帰りたいのですが、持ち運べる量は重量制限$W$によって決まります。アイテム$i$を持ち帰るかどうかの決定を二進変数$x_{i}$で表します。そのアイテムの重量は$w_i$であり、価値は$v_i$です。

まず、これらの値を定義しましょう：

In [14]:
W = jm.Placeholder("W") # The maximum weight we can carry
# The values and weights of each items can represented as
# one-dimensional Placeholders.
values = jm.Placeholder("v", ndim=1) 
weights = jm.Placeholder("w", ndim=1) 
# We can define N in relation to the size of our placeholders,
# so it's calculated automatically based on the instance data.
# The optional "latex" parameter lets us define how this is
# displayed in the LaTeX output.
N = values.len_at(0, latex="N")

# we have N decision variables, which we represent with a 
# one-dimensional BinaryVar. We must specify that it has N elements.
x = jm.BinaryVar("x", shape=(N,))

# an index we'll use for summations
i = jm.Element("i", (0, N))

最後に`Element`を作成します。これらはインデックスとして使用されます。`jijmodeling.sum`を使用すると、シグマ記法に似たスタイルで総和を記述できます。ここでは、このインデックスが0（含む）から$N$（含まない）までの範囲であることを定義します。これを事前に記述するのは奇妙に感じるかもしれませんが、そうすることで再利用が可能になり、全体的に便利です。

この問題では、総価値を最大化しながら、総重量が制限内に収まるようにしたいと考えています。これを次のように記述できます：

In [15]:
problem = jm.Problem("knapsack", sense = jm.ProblemSense.MAXIMIZE)
problem += jm.sum(i, values[i] * x[i])
problem += jm.Constraint("weight limit", jm.sum(i, weights[i] * x[i]) <= W)
problem

念のため、JijModelingの表現は通常のPythonオブジェクトのように変数に格納できます。大規模な問題に取り組む際には、複雑な表現を小さなものから構築することができ、後で理解しやすく、修正しやすくなります。このような小さな問題ではあまり役に立ちませんが、例として、前のコードスニペットを次のように書き直すことができます：

In [16]:
chosen_v = values[i] * x[i]
chosen_w = weights[i] * x[i]
sum_of_values = jm.sum(i, chosen_v)
sum_of_weights = jm.sum(i, chosen_w)
weight_below_limit = sum_of_weights <= W

problem = jm.Problem("knapsack", sense = jm.ProblemSense.MAXIMIZE)
problem += sum_of_values
problem += jm.Constraint("weight limit", weight_below_limit)
problem

これらの2つのモデルは同等です。モデルの記述方法は、好みや便利さの問題です。

### モデルの利用

モデルが完成しましたが、LaTeXにエクスポートする以外にはあまりできることはありません。インスタンスデータと組み合わせて、最適化ソルバーの入力を生成することができます。例として、無料で利用できる[JijModelingTranspiler](https://www.documentation.jijzept.com/docs/jijmodelingtranspiler/)を使用してモデルを変換し、[OpenJij](https://openjij.github.io/OpenJij/index.html)を使用してモデルを解く方法を見てみましょう。なお、[JijZept](https://www.documentation.jijzept.com/docs/jijzept/)で提供されているツールもJijModelingモデルを入力として受け入れます。

これは簡単なデモンストレーションであり、JijModelingTranspilerやOpenJijの機能の完全な紹介ではありません。詳細については、それぞれのドキュメントを参照してください。

JijModelingTranspilerを使用すると、インスタンスデータを渡して中間表現を作成し、さまざまな形式に迅速に変換できますが、今回はQUBOに変換するだけです。

In [17]:
from jijmodeling_transpiler.core import compile_model
from jijmodeling_transpiler.core.pubo import transpile_to_pubo

data = {
 "W": 100,
 "v": [100, 90, 80, 70, 60, 50, 40, 30],
 "w": [1, 5, 10, 20, 30, 40, 50, 60, 70]
}

compiled = compile_model(problem, data)
qubo, _ = transpile_to_pubo(compiled).get_qubo_dict()

これで、特定の最適化ソルバーに有効な入力であるQUBO辞書ができました。このような辞書を手作業で書き出すのは非常にエラーが発生しやすいですが、幸いにもモデルを人間に優しい方法で記述するだけで済み、検証が容易です。この辞書を使用して`openjij`で実際に問題を解くことができます：

In [18]:
import openjij as oj

sampler = oj.SASampler()
response = sampler.sample_qubo(qubo, num_reads=1)
response.first

Sample(sample={0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0, 7: 0}, energy=-5.501111111111113, num_occurrences=1)

これは`openjij`のシミュレーテッドアニーリングサンプラーを使用しており、`num_reads`パラメータは1回だけサンプリングすることを示しています。これを増やしてソルバーを複数回サンプリングし、レスポンスオブジェクトを使用してさまざまな結果を探索することができます。しかし、このサイズの問題では、すべてのサンプルが最適解に到達するため、ここでは1回のサンプルを行い、見つかった「最良」の解を見ています。それは次のようなオブジェクトです：

```
Sample(sample={0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0, 7: 0}, energy=-5.501111111111113, num_occurrences=1)
```

サンプル辞書は、各決定変数に対してソルバーが求めた値を示しています。TranspilerやOpenJijを使用して結果をより良く処理および視覚化したり、同じモデルを異なる目的で再利用したりするために、さらに多くのことができますが、それについてはそれぞれのドキュメントページを参照してください。

## 次のステップ

- [Examples](https://www.documentation.jijzept.com/docs/category/learn)
- [JijZept](https://www.documentation.jijzept.com/docs/jijzept/)
- [JijModelingTranspiler](https://www.documentation.jijzept.com/docs/jijmodelingtranspiler/)